In [29]:
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
from tools.market_data.ftx_market_data import GetFTXData
from tools.market_data.okx_market_data import GetOKXData
from tools.market_data.binance_market_data import GetBinanceData
import numpy as np
import pandas as pd
import requests
from json import JSONDecodeError
import time
from datetime import datetime

In [5]:
symbols_data_url = 'https://api.coingecko.com/api/v3/coins/list'
symbols = requests.get(symbols_data_url).json()

In [10]:
symbols_df = pd.DataFrame(symbols, columns=['id', 'symbol', 'name'])

In [33]:
temp = []
i = 0
for id_num in symbols_df.id:
    symbols_url = f'https://api.coingecko.com/api/v3/coins/{id_num}?localization=false&tickers=false&market_data=true&community_data=false&developer_data=false'
    req = requests.get(symbols_url)
    try:
        resp = req.json()
    except JSONDecodeError:
        print(f'''{datetime.now()} too many requests. waiting for new rate limits. scanned {i}/{len(symbols_df)}. need to wait {req.headers['Retry-After']} s.''')
        time.sleep(int(req.headers['Retry-After']))
        resp = requests.get(symbols_url).json()
    try:
        temp.append([resp['id'], resp['symbol'], resp['name'],  resp['sentiment_votes_up_percentage'], resp['sentiment_votes_down_percentage'], resp['market_cap_rank'], resp['market_data']['current_price']['usd'], resp['market_data']['market_cap']['usd']])
    except KeyError:
        continue
    i += 1

2022-07-03 20:04:05.805459 too many requests. waiting for new rate limits. scanned 61/13372. need to wait 65 s.
2022-07-03 20:05:26.653789 too many requests. waiting for new rate limits. scanned 112/13372. need to wait 65 s.
2022-07-03 20:06:45.868346 too many requests. waiting for new rate limits. scanned 159/13372. need to wait 65 s.
2022-07-03 20:08:12.792316 too many requests. waiting for new rate limits. scanned 226/13372. need to wait 65 s.
2022-07-03 20:09:31.104997 too many requests. waiting for new rate limits. scanned 270/13372. need to wait 65 s.
2022-07-03 20:10:52.892132 too many requests. waiting for new rate limits. scanned 322/13372. need to wait 65 s.
2022-07-03 20:12:19.586898 too many requests. waiting for new rate limits. scanned 386/13372. need to wait 65 s.
2022-07-03 20:13:34.982612 too many requests. waiting for new rate limits. scanned 420/13372. need to wait 65 s.
2022-07-03 20:14:57.478019 too many requests. waiting for new rate limits. scanned 476/13372. nee

In [18]:
#użycie
#  "sentiment_votes_up_percentage": 55.49,
#  "sentiment_votes_down_percentage": 44.51,
# ^ ww. metryki są dostępne w api

In [39]:
df = pd.DataFrame(temp, columns=['id', 'symbol', 'name', 'positive_sentiment', 'negative_sentiment', 'market_cap_rank', 'market_cap', 'price'])

In [41]:
df.head()

,id,symbol,name,positive_sentiment,negative_sentiment,market_cap_rank,market_cap,price
0,01coin,zoc,01coin,NaN,NaN,NaN,0.000153,0.0
1,0-5x-long-algorand-token,algohalf,0.5X Long Algorand,NaN,NaN,NaN,10291.950000,0.0
2,0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index,NaN,NaN,NaN,13667.760000,0.0
3,0-5x-long-ascendex-token-token,asdhalf,0.5X Long AscendEx,NaN,NaN,NaN,9500.780000,0.0
4,0-5x-long-bitcoin-cash-token,bchhalf,0.5X Long Bitcoin Cash,NaN,NaN,NaN,6381.290000,0.0


In [40]:
df.to_csv('coingecko_symbols.csv')